Install R + rpy2

In [2]:
!apt-get install -y r-base
!pip install rpy2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
r-base is already the newest version (4.5.2-1.2204.0).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


Enable R

In [3]:
%reload_ext rpy2.ipython

Install Packages

In [5]:
%%R
install.packages("kernlab")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/kernlab_0.9-33.tar.gz'
Content type 'application/x-gzip' length 1025337 bytes (1001 KB)
downloaded 1001 KB


The downloaded source packages are in
	‘/tmp/RtmpCqCkiE/downloaded_packages’


In [6]:
%%R
install.packages("kknn")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependency ‘igraph’

trying URL 'https://cran.rstudio.com/src/contrib/igraph_2.2.1.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/kknn_1.4.1.tar.gz'

The downloaded source packages are in
	‘/tmp/RtmpCqCkiE/downloaded_packages’


# Question 1

clean and load the data


In [8]:
%%R

library(kernlab)

data <- read.table("credit_card_data-headers.txt", header = TRUE)
dim(data)
head(data)

  A1    A2    A3   A8 A9 A10 A11 A12 A14 A15 R1
1  1 30.83 0.000 1.25  1   0   1   1 202   0  1
2  0 58.67 4.460 3.04  1   0   6   1  43 560  1
3  0 24.50 0.500 1.50  1   1   0   1 280 824  1
4  1 27.83 1.540 3.75  1   0   5   0 100   3  1
5  1 20.17 5.625 1.71  1   1   0   1 120   0  1
6  1 32.08 4.000 2.50  1   1   0   0 360   0  1


Split predictors and response

In [9]:
%%R

X <- as.matrix(data[, 1:10])
y <- as.factor(data[, 11])

Train linear SVM

In [11]:
%%R

model <- ksvm(
  X, y,
  type = "C-svc",
  kernel = "vanilladot",
  C = 10,
  scaled = TRUE
)

 Setting default kernel parameters  


Extract classifier coefficients

In [15]:
%%R

a <- colSums(model@xmatrix[[1]] * model@coef[[1]])
a0 <- -model@b
print(a)
print(a0)

           A1            A2            A3            A8            A9 
-0.0009033671 -0.0007891036 -0.0016972133  0.0026113629  1.0050221405 
          A10           A11           A12           A14           A15 
-0.0028363016 -0.0001569285 -0.0003925964 -0.0012784443  0.1064387167 
[1] 0.08157559


Print Accuracy

In [13]:
%%R

pred <- predict(model, X)
mean(pred == y)

[1] 0.8639144


#Question 2

RBF (Gaussian) kernel SVM

In [17]:
%%R

model_rbf <- ksvm(
  X, y,
  type = "C-svc",
  kernel = "rbfdot",
  C = 10,
  scaled = TRUE
)

Return predictions and accuracy

In [18]:
%%R

pred_rbf <- predict(model_rbf, X)
mean(pred_rbf == y)

[1] 0.9204893


#Question 3

Load package

In [19]:
%%R

library(kknn)

Convert response to numeric for kknn, chosen value of k is 7

In [22]:
%%R

y_num <- as.numeric(as.character(y))

n <- nrow(data)
pred_knn <- numeric(n)

k <- 7

for (i in 1:n) {
  model_knn <- kknn(
    y_num[-i] ~ .,
    train = data[-i, 1:10],
    test  = data[i, 1:10],
    k = k,
    scale = TRUE
  )
  # kknn returns fraction of neighbors equal to 1
  prob <- fitted(model_knn)

  # classification rule
  pred_knn[i] <- ifelse(prob >= 0.5, 1, 0)
}

Show Accuracy

In [23]:
%%R
mean(pred_knn == y_num)

[1] 0.8470948
